In [1]:
# Загрузка и запуск модели из ultralytics
!pip install ultralytics
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
results = model("https://ultralytics.com/images/bus.jpg")
print(results)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

100%|██████████| 6.23M/6.23M [00:00<00:00, 106MB/s]


100%|██████████| 476k/476k [00:00<00:00, 16.7MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 112.0ms
Speed: 12.7ms preprocess, 112.0ms inference, 952.5ms postprocess per image at shape (1, 3, 640, 480)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'ban

In [2]:
# Конвертация модели в ONNX
!pip install onnx onnxruntime
import onnx
import onnxruntime
import torch.onnx
#torch.onnx.export(model, "bus.jpg", "yolov8.onnx") - пришлось отказаться от torch.onnx.export из-за ошибки
success = model.export(format='onnx')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.7 MB/s eta 0:00:00
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 0.9s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (2.6s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


In [3]:
from PIL import Image
img = Image.open("bus.jpg")
img_width, img_height = img.size;
img = img.resize((640,640))
img = img.convert("RGB");

In [4]:
import numpy as np
input = np.array(img)
input = input.transpose(2,0,1)
input = input.reshape(1,3,640,640)
input = input.astype(np.float32)

In [5]:
!ls /content

bus.jpg  sample_data  yolov8n.onnx  yolov8n.pt


In [6]:
# Загрузка и запуск модели в ONNX
onnx_model_path='/content/yolov8n.onnx'
onnx_model=onnx.load(onnx_model_path)
ort_session = onnxruntime.InferenceSession(onnx_model_path)
ort_inputs = {ort_session.get_inputs()[0].name: input}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели Yolo8 в ONNX:", ort_outs)

Результат запуска модели Yolo8 в ONNX: [array([[[     6.6988,      10.805,      25.342, ...,      535.39,      548.06,      561.05],
        [     3.3887,      2.9973,      3.4545, ...,      566.58,      576.25,      605.07],
        [     12.244,      17.849,      21.825, ...,      210.03,      183.66,      158.33],
        ...,
        [          0,           0,           0, ...,  2.9802e-08,  2.9802e-07,  1.2517e-06],
        [          0,           0,           0, ...,  9.8348e-07,  1.9372e-06,  3.2485e-06],
        [          0,           0,           0, ...,  4.7684e-07,   1.967e-06,  4.5002e-06]]], dtype=float32)]


In [7]:
# Обратная конвертация и запуск
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert(onnx_model_path)
input = torch.from_numpy(input)
output = torch_model(input)
print(output)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.2 MB/s eta 0:00:00
tensor([[[6.6989e+00, 1.0805e+01, 2.5342e+01,  ..., 5.3539e+02, 5.4806e+02, 5.6105e+02],
         [3.3887e+00, 2.9973e+00, 3.4545e+00,  ..., 5.6658e+02, 5.7625e+02, 6.0507e+02],
         [1.2244e+01, 1.7849e+01, 2.1825e+01,  ..., 2.1003e+02, 1.8366e+02, 1.5833e+02],
         ...,
         [3.2687e-15, 2.7802e-16, 1.2164e-14,  ..., 4.3387e-08, 3.0332e-07, 1.2041e-06],
         [1.9300e-09, 9.4084e-11, 4.5921e-09,  ..., 9.7548e-07, 1.9234e-06, 3.2217e-06],
         [1.3999e-11, 1.8535e-12, 3.3871e-11,  ..., 5.0513e-07, 2.0161e-06, 4.4900e-06]]], grad_fn=<CatBackward0>)
